# Глава 7: Создание чат-приложений
## Быстрый старт с Azure OpenAI API


## Обзор
Этот блокнот адаптирован из [репозитория примеров Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), который включает блокноты, также использующие сервис [OpenAI](notebook-openai.ipynb).

Python API OpenAI работает и с Azure OpenAI, с некоторыми изменениями. Узнайте больше о различиях здесь: [Как переключаться между конечными точками OpenAI и Azure OpenAI с помощью Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

Для получения дополнительных примеров быстрого старта обратитесь к официальной [документации быстрого старта Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)


## Содержание  

[Обзор](../../../../07-building-chat-applications/python)  
[Начало работы с Azure OpenAI Service](../../../../07-building-chat-applications/python)  
[Создайте свой первый запрос](../../../../07-building-chat-applications/python)  

[Сценарии использования](../../../../07-building-chat-applications/python)    
[1. Суммирование текста](../../../../07-building-chat-applications/python)  
[2. Классификация текста](../../../../07-building-chat-applications/python)  
[3. Генерация новых названий продуктов](../../../../07-building-chat-applications/python)  
[4. Тонкая настройка классификатора](../../../../07-building-chat-applications/python)  
[5. Встраивания](../../../../07-building-chat-applications/python)

[Ссылки](../../../../07-building-chat-applications/python)


### Начало работы с Azure OpenAI Service

Новым клиентам необходимо [подать заявку на доступ](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst) к Azure OpenAI Service.  
После одобрения клиенты могут войти в портал Azure, создать ресурс Azure OpenAI Service и начать экспериментировать с моделями через студию  

[Отличный ресурс для быстрого начала работы](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### Создайте свой первый запрос  
Это короткое упражнение даст базовое введение в отправку запросов модели OpenAI для простой задачи «суммирования».


**Шаги**:  
1. Установите библиотеку OpenAI в вашу среду Python  
2. Загрузите стандартные вспомогательные библиотеки и установите ваши обычные учетные данные безопасности OpenAI для созданного вами сервиса OpenAI  
3. Выберите модель для вашей задачи  
4. Создайте простой запрос для модели  
5. Отправьте ваш запрос в API модели!


### 1. Установите OpenAI


  > [!ПРИМЕЧАНИЕ] Этот шаг не обязателен, если запускать этот блокнот в Codespaces или внутри Devcontainer


In [ ]:
%pip install openai python-dotenv

### 2. Импортировать вспомогательные библиотеки и создать учетные данные


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. Поиск подходящей модели  
Модели GPT-3.5-turbo или GPT-4 могут понимать и генерировать естественный язык. Сервис предлагает четыре возможности моделей, каждая из которых обладает разным уровнем мощности и скорости, подходящими для различных задач. 

[Azure OpenAI models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. Проектирование подсказок  

«Магия больших языковых моделей заключается в том, что, обучаясь минимизировать ошибку предсказания на огромных объемах текста, модели в итоге усваивают концепции, полезные для этих предсказаний. Например, они изучают такие концепции, как»(1):

* как правильно писать
* как работает грамматика
* как перефразировать
* как отвечать на вопросы
* как вести разговор
* как писать на многих языках
* как программировать
* и т.д.

#### Как управлять большой языковой моделью  
«Из всех входных данных для большой языковой модели самым влиятельным, безусловно, является текстовая подсказка»(1).

Большие языковые модели можно стимулировать к генерации вывода несколькими способами:

- Инструкция: скажите модели, чего вы хотите
- Завершение: побудите модель завершить начало того, что вы хотите
- Демонстрация: покажите модели, чего вы хотите, с помощью:
- Нескольких примеров в подсказке
- Сотен или тысяч примеров в обучающем наборе для дообучения



#### Существует три основных правила создания подсказок:

**Показывайте и объясняйте**. Четко указывайте, чего вы хотите, через инструкции, примеры или их комбинацию. Если вы хотите, чтобы модель упорядочила список элементов в алфавитном порядке или классифицировала абзац по настроению, покажите ей, что именно вы хотите.

**Предоставляйте качественные данные**. Если вы пытаетесь создать классификатор или заставить модель следовать определенному шаблону, убедитесь, что примеров достаточно. Обязательно вычитывайте свои примеры — модель обычно достаточно умна, чтобы распознать базовые орфографические ошибки и дать ответ, но она также может предположить, что это сделано намеренно, и это может повлиять на ответ.

**Проверяйте настройки.** Параметры temperature и top_p контролируют, насколько детерминированной будет модель при генерации ответа. Если вы запрашиваете ответ, где есть только один правильный вариант, лучше установить эти параметры ниже. Если вы хотите получить более разнообразные ответы, установите их выше. Самая распространенная ошибка — считать, что эти параметры управляют «умом» или «креативностью» модели.


Источник: https://learn.microsoft.com/azure/ai-services/openai/overview


изображение создаёт ваш первый текстовый запрос!


### 5. Отправить!


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### Повторите тот же вызов, как сравниваются результаты?


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Суммировать текст  
#### Задача  
Суммируйте текст, добавив «tl;dr:» в конце отрывка текста. Обратите внимание, как модель понимает, как выполнять ряд задач без дополнительных инструкций. Вы можете экспериментировать с более описательными подсказками, чем tl;dr, чтобы изменить поведение модели и настроить получаемое резюме(3).  

Недавние исследования продемонстрировали значительные улучшения во многих задачах и бенчмарках NLP благодаря предварительному обучению на большом корпусе текста с последующей донастройкой на конкретную задачу. Хотя архитектура обычно не зависит от задачи, этот метод все же требует специализированных наборов данных для донастройки, состоящих из тысяч или десятков тысяч примеров. В отличие от этого, люди обычно могут выполнять новую языковую задачу, имея всего несколько примеров или простые инструкции — то, с чем современные системы NLP по-прежнему в значительной степени испытывают трудности. Здесь мы показываем, что масштабирование языковых моделей значительно улучшает универсальную работу с небольшим количеством примеров, иногда даже достигая конкурентоспособности с предыдущими передовыми методами донастройки.  

Tl;dr


# Упражнения для нескольких вариантов использования  
1. Суммирование текста  
2. Классификация текста  
3. Генерация новых названий продуктов  
4. Встраивания  
5. Тонкая настройка классификатора


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Классификация текста  
#### Задача  
Классифицировать элементы по категориям, предоставленным во время вывода. В следующем примере мы указываем как категории, так и текст для классификации в подсказке (*playground_reference). 

Запрос клиента: Здравствуйте, недавно одна из клавиш на клавиатуре моего ноутбука сломалась, и мне нужна замена:

Классифицированная категория:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Создание новых названий продуктов
#### Задача
Создайте названия продуктов из примерных слов. В подсказке мы включаем информацию о продукте, для которого собираемся сгенерировать названия. Мы также предоставляем похожий пример, чтобы показать желаемый шаблон. Значение температуры установлено высоко, чтобы увеличить случайность и получить более инновационные ответы.

Описание продукта: Домашний миксер для молочных коктейлей  
Исходные слова: быстрый, здоровый, компактный.  
Названия продуктов: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Описание продукта: Пара обуви, которая подходит для любого размера ноги.  
Исходные слова: адаптируемый, подходящий, универсальный.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## Встраивания
В этом разделе показано, как получать встраивания и находить сходства между словами, предложениями и документами. Чтобы запустить следующие блокноты, необходимо развернуть модель, использующую `text-embedding-ada-002` в качестве базовой модели, и указать имя её развертывания в файле .env, используя переменную `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT`.


### Таксономия моделей - Выбор модели встраивания

**Таксономия моделей**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (семейство встраиваний)  
{capability} --> ada             (все остальные модели встраивания будут сняты с поддержки в 2024 году)  
{input-type} --> n/a             (указывается только для моделей поиска)  
{identifier} --> 002             (версия 002)  

model = 'text-embedding-ada-002'


  > [!ПРИМЕЧАНИЕ] Следующий шаг не обязателен, если запускать этот блокнот в Codespaces или внутри Devcontainer


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## Сравнение статьи из набора данных cnn daily news
источник: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# References  
- [Документация Azure - Модели Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [Примеры OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  


# Для дополнительной помощи  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Contributors
* Луи Ли  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, имейте в виду, что автоматический перевод может содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется обращаться к профессиональному переводу, выполненному человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
